In [1]:
import sys
import json
sys.path.append('..')
sys.path.append('../questions_construction')

from src.common import *
from model_performances import *
from copy import deepcopy

import pandas as pd

EVERYTHING_POSTFIX = 'everything'

In [5]:
data_all_flat = []
for ramifications in [WITH_RAMIFICATIONS, WITH_RAMIFICATIONS]:
    for answer_response in [TRUE_FALSE_ANSWER, FREE_ANSWER]:
        for model_name in PROMPT_MODEL_NAMES:
            for prompt_type in PROMPT_TYPES:
                try:
                    results_path = os.path.join(STATISTICS_PATH, ramifications, model_name, prompt_type,f'{answer_response}.json')
                    with open(results_path) as f:
                        data = json.load(f)
                    data_all_flat.extend(data)
                except Exception:
                    pass

In [6]:
data_all_flat[0]

{'true_false_answer_by_category': {'object_tracking': {'1': 0.5,
   '5': 0.5,
   '10': 1.0,
   '15': 0.5,
   '19': 0.5,
   'all': 0.6},
  'fluent_tracking': {'1': 1.0,
   '5': 0.5,
   '10': 0.75,
   '15': 0.25,
   '19': 0.5,
   'all': 0.6},
  'state_tracking': {'1': 1.0,
   '5': 0.5,
   '10': 1.0,
   '15': 1.0,
   '19': 0.0,
   'all': 0.7},
  'action_executability': {'1': 0.5,
   '5': 0.5,
   '10': 0.5,
   '15': 1.0,
   '19': 1.0,
   'all': 0.7},
  'effects': {'1': 0.5, '5': 0.5, '10': 0.5, '15': 1.0, '19': 1.0, 'all': 0.7},
  'numerical_reasoning': {'1': 0.75,
   '5': 0.5,
   '10': 1.0,
   '15': 0.0,
   '19': 0.0,
   'all': 0.7272727272727273},
  'all': {'1': 0.75,
   '5': 0.5,
   '10': 1.0,
   '15': 0.0,
   '19': 0.0,
   'all': 0.7272727272727273}},
 'true_false_answer_in_response_by_category': {'object_tracking': {'1': 0.5,
   '5': 0.5,
   '10': 1.0,
   '15': 0.5,
   '19': 0.5,
   'all': 0.6},
  'fluent_tracking': {'1': 1.0,
   '5': 0.5,
   '10': 0.75,
   '15': 0.25,
   '19': 0.5,
 

In [ ]:
def fish_out(data_all_flat, keys_values_ls):
    results = []
    for d in data_all_flat:
       if all(d[k]==v for k, v in keys_values_ls):
           results.append(d)
    return results
            

In [ ]:
plan_length = 1
index = []
for category in QUESTION_CATEGORIES + [ALL_CATEGORIES]:
    for ramifications in RAMIFICATION_TYPES:
        for answer_response in ANSWER_RESPONSES:
            index.append([plan_length, category, ramifications, answer_response])
            data_columns = {}
            for model_name in PROMPT_MODEL_NAMES:
                for prompt_type in PROMPT_TYPES:
                    keys_values_ls = [(CATEGORY_KEY, category), (RAMIFICATION_KEY, ramifications), (ANSWER_RESPONSE_KEY, answer_response), (MODEL_NAME_KEY, model_name), (PROMPT_TYPE_KEY, prompt_type), (PLAN_LENGTH_KEY, plan_length)]
                    data_columns[(model_name, prompt_type)] = fish_out(data_all_flat, keys_values_ls)
                    


In [3]:
import pandas as pd

data = [{'b': 2, 'c': 3}, {'a': 10, 'b': 20, 'c': 30}]

df = pd.DataFrame(data, index=['first', 'second'])

[{'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'blocksworld'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'depots'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'driverlog'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'goldminer'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'grippers'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'logistics'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_name': 'llama-7b',
  'eval_type': 'zero_shot_data',
  'domain': 'miconic'},
 {'true_false_answer': {},
  'free_answer': {},
  'model_na